In [1]:
!pip install beautifulsoup4

In [5]:
!pip install lxml

In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [15]:
years =[1930, 1934, 1938, 1950, 1954, 
        1958, 1962, 1966, 1970, 1974, 
        1978, 1982, 1986, 1990, 1994, 
        1998, 2002, 2006, 2010, 2014, 2018]

In [11]:
#create a function to get wc results from all the different years
def get_matches(year):
    #use fstring to automatically get the correct url for the year input 
    url = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    #response
    r = requests.get(url)
    #print(r.text) #get text of r
    content = r.text
    soup = BeautifulSoup(content, 'lxml') #lxml is a parser that allows data to be extracted

    
    
    #returns a list of elements with the div tag and footballbox class
    matches = soup.find_all('div', class_='footballbox')
    
    
    
    home = []
    score = []
    away = []
    
    for match in matches: 
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    #create a dictionary to store the scores
    dict_football = {'home': home, 'score': score, 'away': away}

    #store the data in a more organised manner in a dataframe 
    df_football = pd.DataFrame(dict_football)
    #print results from the year input
    df_football['year'] =year
    return df_football

In [13]:
get_matches(2022)

,home,score,away,year
0,Qatar,0–2,Ecuador,2022
1,Senegal,0–2,Netherlands,2022
2,Qatar,1–3,Senegal,2022
3,Netherlands,1–1,Ecuador,2022
4,Ecuador,1–2,Senegal,2022
...,...,...,...,...
59,England,1–2,France,2022
60,Argentina,3–0,Croatia,2022
61,France,2–0,Morocco,2022
62,Croatia,2–1,Morocco,2022


In [16]:
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_wc_historical_data.csv', index = False)

In [17]:
df_ongoing = get_matches(2022)
df_ongoing.to_csv('fifa_wc_ongoing.csv', index=False)

### Scraping missing data  

In [18]:
!pip install selenium

     |████████████████████████████████| 6.3 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 384 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.0 MB/s  eta 0:00:01


In [28]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.common.exceptions import WebDriverException
import time 

In [45]:
path = 'Users/suhyun/Downloads/chromedriver'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

In [47]:
def get_missing_data(year):
    url = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    driver.get(url)
    matches = driver.find_elements(by='xpath', value='//td[@align="right"]/.. | //td[@style="text-align:right;"]/..')
    
    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football

In [48]:
fifa = [get_missing_data(year) for year in years]
driver.quit()
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_wc_missing_data.csv", index=False)

## Data Cleaning 

In [88]:
df_historical_data = pd.read_csv('fifa_wc_historical_data.csv')
df_ongoing = pd.read_csv('fifa_wc_ongoing.csv')
df_missing_data = pd.read_csv('fifa_wc_missing_data.csv')

### clean df_ongoing

In [89]:
#.strip to remove trailing and blank spaces in the data 
df_ongoing['home'] = df_ongoing['home'].str.strip()
df_ongoing['away'] = df_ongoing['away'].str.strip()

### clean df_missing_data and add to add_historical_data

In [90]:
#remove null data 
df_missing_data.dropna(inplace=True)

#add missing data to historical data 
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
#sort data by year in asc order
df_historical_data.sort_values('year', inplace=True)
df_historical_data




,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
600,France,4–2,Croatia,2018
599,Belgium,2–0,England,2018
598,Croatia,2–1 (a.e.t.),England,2018
596,Russia,2–2 (a.e.t.),Croatia,2018


### clean df_historical data

In [91]:
# deleting match with walk over (one of the countries dropped out before the match)
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
#gets the matches that do not have a digit and - format e.g normal: 2-1 not desirable: 2-1 (a.e.t)
#regex=True will replace everything that is not a digit and - e.g (a.e.t) with ''
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
600,France,Croatia,2018,4,2,6
599,Belgium,England,2018,2,0,2
598,Croatia,England,2018,2,1,3
596,Russia,Croatia,2018,2,2,4


### Export clean dataframes

In [92]:
df_historical_data.to_csv('clean_fifa_wc_historical_data.csv',index=False)
df_ongoing.to_csv('clean_fifa_wc_ongoing.csv',index=False)

### predict the wc winner for 2022

In [93]:
import pickle
from scipy.stats import poisson

In [94]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_wc_historical_data.csv')
df_ongoing = pd.read_csv('clean_fifa_wc_ongoing.csv')

In [95]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [96]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

### GROUP STAGE

In [97]:
#GROUP STAGE 
df_fixture_group_48 = df_ongoing[:48].copy()
df_fixture_knockout = df_ongoing[48:56].copy()
df_fixture_quarter = df_ongoing[56:60].copy()
df_fixture_semi = df_ongoing[60:62].copy()
df_fixture_final = df_ongoing[62:].copy()

In [98]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [99]:
dict_table['Group A']

,Team,Pts
0,Netherlands,11.0
1,Senegal,8.0
2,Ecuador,6.0
3,Qatar (H),0.0


In [100]:
df_fixture_knockout

,home,score,away,year
48,Netherlands,3–1,United States,2022
49,Argentina,2–1,Australia,2022
50,France,3–1,Poland,2022
51,England,3–0,Senegal,2022
52,Japan,1–1 (a.e.t.),Croatia,2022
53,Brazil,4–1,South Korea,2022
54,Morocco,0–0 (a.e.t.),Spain,2022
55,Portugal,6–1,Switzerland,2022


### KNOCKOUT

In [101]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022,?
49,Argentina,2–1,Australia,2022,?
50,France,3–1,Poland,2022,?
51,England,3–0,Senegal,2022,?
52,Japan,1–1 (a.e.t.),Croatia,2022,?
53,Brazil,4–1,South Korea,2022,?
54,Morocco,0–0 (a.e.t.),Spain,2022,?
55,Portugal,6–1,Switzerland,2022,?


In [102]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [103]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022,Netherlands
49,Argentina,2–1,Australia,2022,Argentina
50,France,3–1,Poland,2022,France
51,England,3–0,Senegal,2022,England
52,Japan,1–1 (a.e.t.),Croatia,2022,Croatia
53,Brazil,4–1,South Korea,2022,Brazil
54,Morocco,0–0 (a.e.t.),Spain,2022,Spain
55,Portugal,6–1,Switzerland,2022,Portugal


### QUARTER FINAL

In [104]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [105]:
update_table(df_fixture_knockout, df_fixture_quarter)


,home,score,away,year,winner
56,Croatia,1–1 (a.e.t.),Brazil,2022,?
57,Netherlands,2–2 (a.e.t.),Argentina,2022,?
58,Morocco,1–0,Portugal,2022,?
59,England,1–2,France,2022,?


### SEMI FINAL

In [106]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Argentina,3–0,Croatia,2022,?
61,France,2–0,Morocco,2022,?


In [107]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Argentina,3–0,Croatia,2022,Argentina
61,France,2–0,Morocco,2022,France


### FINAL

In [108]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Croatia,2–1,Morocco,2022,?
63,Argentina,Match 64,France,2022,?


In [109]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Croatia,2–1,Morocco,2022,Croatia
63,Argentina,Match 64,France,2022,France


## And the winner of the 2022 FIFA WORLD CUP IS....

In [113]:
print(get_winner(df_fixture_final).loc[63])

home      Argentina
score      Match 64
away         France
year           2022
winner       France
Name: 63, dtype: object
